In [ ]:
from datetime import datetime

import numpy as np

import os

import pandas as pd

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model

from time import time, gmtime, strftime

from bert.tokenization.bert_tokenization import FullTokenizer

from utils.models.modelUtils import loadMeanFromFile, calcAccConcatModel

from final_models import create_image_inceptionv3_model, create_text_model, create_model_meta, buildConcatModelImageTitleMeta

from utils.callbacks.MyCallbacks import MyCallbacks
from utils.datagenUtils.three_modal.DataSequenceImageTextMeta import DataSequenceImageTextMeta
from utils.datagenUtils.datagenUtils import checkDataframe

from utils.telegramUtils.telegram_bot import telegram_send_message
from utils.textUtils.textpreprocessing import FakeDetectionDataTrainVal, FakeDetectionDataTest
from utils.textUtils.commentsProcessing import FakeDetectionDataCommentsTest, FakeDetectionDataCommentsTrainVal

from utils.callbacks.callbackUtils import plotTimesPerEpoch

from utils.fileDirUtils.fileDirUtils import createDirIfNotExists

from sklearn.metrics import accuracy_score


In [ ]:
#Verbose settings:
verbose = False
TF_VERBOSE = 1 # 1 = Progress bar 2 = one line per epoch only!
TF_DETERMINISTIC_OPS = 1 # Makes everything also on GPU deterministic

# Classes:
NUM_CLASS = 2  # FAKE | NO FAKE

# Hyperparameters
GLOBAL_BATCH_SIZE = 96
EPOCHS = 6

# Optimizer parameters:
# Adam
LEARNING_RATE = 1e-5
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-8

#optimizers:

optimizer = Adam(LEARNING_RATE, BETA_1, BETA_2, EPSILON)

# Bert Parameters
MAX_SEQUENCE_LENGTH = 128 # from model definition!

# Image Model  Parameters
IMG_WIDTH = 768
IMG_HEIGHT = 768
IMG_DEPTH = 3
IMG_SIZES = (IMG_WIDTH, IMG_HEIGHT)

# Training switches
RETRAIN_WHOLE_MODEL = False
loadPretrained = True # Attention that model weights path is set !

# Custom telegram send text 
CUSTOM_TEXT = f'Batch Size: {GLOBAL_BATCH_SIZE}, Epochs: {EPOCHS}, Optimizer: Adam, Learning Rate; {LEARNING_RATE}, Beta_1: {BETA_1}, Beta_2: {BETA_2}, Epsilon: {EPSILON}, Image Sizes: {IMG_SIZES}'


telegram_send_message(f'-----------------START-----------------')
print('START')
print(CUSTOM_TEXT)
telegram_send_message(CUSTOM_TEXT)

In [ ]:
pathToRootModelDir = '/home/armin/repos/fkd-model-handling/models/best_models'

pathToBertModelTitle = os.path.join(pathToRootModelDir, 'single_text_title', 'weights-improvement-02-0.88.hdf5')
pathToBertModelComments = os.path.join(pathToRootModelDir, 'single_text_comments', 'weights-improvement-03-0.87.hdf5')
pathToImageModel = os.path.join(pathToRootModelDir, 'single_visual', 'weights-improvement-02-0.81.hdf5')
pathToMetaModel = os.path.join(pathToRootModelDir, 'single_meta', 'weights-improvement-100-0.62.hdf5')

pathToImagesTrain = '/home/armin/repos/FKD-Dataset/006_images_resized_2/train/' 
pathToCSVWithFileNamesAndLabelsTrain = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/train_text_image_meta_label.csv'

pathToImagesVal = '/home/armin/repos/FKD-Dataset/006_images_resized_2/val/' 
pathToCSVWithFileNamesAndLabelsVal = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/val_text_image_meta_label.csv'

pathToImagesTest = '/home/armin/repos/FKD-Dataset/006_images_resized_2/test/' 
pathToCSVWithFileNamesAndLabelsTest = '/home/armin/repos/FKD-Dataset/008_text_image_meta_label/test_text_image_meta_label.csv'

checkpointDir = '/home/armin/repos/FKD-Dataset/011_checkpoints'

pathToMeans = '/home/armin/repos/FKD-Dataset/010_configs/means_resized_768.txt'

titleCommentsVisualMetaModelWeightsPath = "/home/armin/repos/FKD-Dataset/011_checkpoints/model_concat_title_comments_visual_meta_2020-07-22_07:48/weights-improvement-04-0.93.hdf5"

root = '/home/armin/repos/fkd-model-handling/'
bert_model_dir = os.path.join(root, 'multi_cased_L-12_H-768_A-12')
bert_config_file = os.path.join(bert_model_dir, "bert_config.json")
bert_ckpt_file = os.path.join(bert_model_dir, "bert_model.ckpt") 

In [ ]:
# Other ettings

# Time settings:
current_time = datetime.now().strftime("%Y-%m-%d_%H:%M")

#Checkpoint settings:
checkpoint_name = "model_concat_title_comments_visual_meta"

checkpointDir = os.path.join(checkpointDir, (checkpoint_name + '_' + current_time))

fileName="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
filePath = os.path.join(checkpointDir, fileName)

In [ ]:
# Callback Handling:
tensorboardDir = os.path.join(checkpointDir, 'tensorboard')

createDirIfNotExists(tensorboardDir)
createDirIfNotExists(checkpointDir)


callbacks_list = MyCallbacks(tensorboardDir, filePath, earlyStopping=True).createCheckpoints()

In [ ]:
df_train_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsTrain, header=0, sep='\t')
df_test_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsTest, header=0, sep='\t')
df_val_ = pd.read_csv(pathToCSVWithFileNamesAndLabelsVal, header=0, sep='\t')

df_train_['2_way_label'] = df_train_['2_way_label'].apply(lambda x: np.array(x))
df_test_['2_way_label'] = df_test_['2_way_label'].apply(lambda x: np.array(x))
df_val_['2_way_label'] = df_val_['2_way_label'].apply(lambda x: np.array(x))

df_train = df_train_
df_test = df_test_
df_val = df_val_

# df_train = df_train[:5000]
# df_test = df_test[:5000]
# df_val = df_val[:5000]

df_train = checkDataframe(df_train, GLOBAL_BATCH_SIZE)
df_val = checkDataframe(df_val, GLOBAL_BATCH_SIZE)
df_test = checkDataframe(df_test, GLOBAL_BATCH_SIZE)


In [ ]:
tokenizer = FullTokenizer(vocab_file=os.path.join(bert_model_dir, "vocab.txt"))

all_title_data = FakeDetectionDataTrainVal(df_train, df_val, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)

all_title_test = FakeDetectionDataTest(df_test, tokenizer, [0,1], MAX_SEQUENCE_LENGTH)



train_title_x = all_title_data.train_x
train_title_val_x = all_title_data.val_x
test_title_x = all_title_test.test_x

In [ ]:
meansOfDataset = loadMeanFromFile(pathToMeans, verbose)

train_seq = DataSequenceImageTextMeta(df_train, pathToImagesTrain, train_title_x, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)

test_seq = DataSequenceImageTextMeta(df_test, pathToImagesTest, test_title_x, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)

val_seq = DataSequenceImageTextMeta(df_val,pathToImagesVal, train_title_val_x, GLOBAL_BATCH_SIZE, IMG_SIZES, meansOfDataset)

STEP_SIZE_TRAIN = len(df_train) // GLOBAL_BATCH_SIZE
STEP_SIZE_TEST = len(df_test) // GLOBAL_BATCH_SIZE
STEP_SIZE_VAL = len(df_val) // GLOBAL_BATCH_SIZE

In [ ]:
mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    # Preparing:
    model_bert_title, model_bert_title_custom_layer = create_text_model(max_seq_len=MAX_SEQUENCE_LENGTH, bert_ckpt_file=bert_ckpt_file, bert_config_file= bert_config_file,NUM_CLASS=NUM_CLASS, overwriteLayerAndEmbeddingSize=False, isPreTrained=True,  pathToBertModelWeights=pathToBertModelTitle, isTrainable=RETRAIN_WHOLE_MODEL)
    model_image, model_image_custom_layer = create_image_inceptionv3_model(NUM_CLASS=NUM_CLASS,isPreTrained=True,pathToInceptionV3ModelWeights=pathToImageModel, isTrainable=RETRAIN_WHOLE_MODEL)
    model_meta, model_meta_custom_layer = create_model_meta(NUM_CLASS, 2, True, pathToMetaModel, isTrainable=RETRAIN_WHOLE_MODEL)
    
    
    
    # Handling same layer name error:
    for i, layer in enumerate(model_bert_title.layers):
        layer._name = layer._name + '_title' # Consider the _ for the setter
    
    model_concat = buildConcatModelImageTitleMeta(model_bert_title, model_image, model_meta, 2)
    
    if loadPretrained:
        model_concat.load_weights(titleCommentsVisualMetaModelWeightsPath)

    if verbose:
        print('--------------Text Title MODEL --------------')
        model_bert_title.summary()
        print('--------------Text Comments MODEL --------------')
        model_image.summary()
        print('--------------model_meta MODEL --------------')
        model_meta.summary()
        print('--------------CONCAT MODEL --------------')
        model_concat.summary()
        for l in model_concat.layers:
            print(l.name, l.trainable)
    
    
    model_concat.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
          optimizer=optimizer,
          metrics=['accuracy'])
    
    history = model_concat.fit(train_seq,
                    steps_per_epoch=STEP_SIZE_TRAIN,epochs=EPOCHS, 
                    validation_data=val_seq,
                    validation_steps=STEP_SIZE_VAL, 
                    callbacks=callbacks_list,
                    use_multiprocessing=False,
                    verbose=TF_VERBOSE,
                    workers=12
                   )

In [ ]:
# for i, layer in enumerate(model_bert_title.layers):
#     layer._name = layer._name + '_title' # Consider the _ for the setter
        
# for i, layer in enumerate(model_bert_title.layers):
#     print(layer.name)

In [ ]:
%%capture

plot_model(model_concat, to_file=os.path.join(checkpointDir,'multiple_inputs_image_text_meta_pretrained.png'), show_shapes=True, dpi=600, expand_nested=False)
plot_model(model_concat, to_file=os.path.join(checkpointDir,'multiple_inputs_image_text_meta_pretrained_nested.png'), show_shapes=True, dpi=600, expand_nested=True)

In [ ]:
# For later model loading
# modelFile = '/home/armin/repos/FKD-Dataset/011_checkpoints/model_concat_title_comments_visual_meta_2020-07-19_14:04/weights-improvement-04-0.95.hdf5'

# model_concat.load_weights(modelFile)

In [ ]:
plotTimesPerEpoch(callbacks_list)

In [ ]:
calcAccConcatModel(model_concat, val_seq, GLOBAL_BATCH_SIZE, 'Val')
calcAccConcatModel(model_concat, test_seq, GLOBAL_BATCH_SIZE, 'Test')

In [ ]:
telegram_send_message(f'-----------------DONE-----------------')
print('done')